In [ ]:
import pandas as pd
import numpy as np
with open("cardio_train.csv") as file:
    df = pd.read_csv(file, delimiter=";")

In [ ]:
df["BMI"] = df.weight / ((df.height/100)**2)

In [ ]:
df["c_ap_hi"] = ""
df["c_ap_lo"] = ""
df["c_weight"] = ""

In [ ]:
for i, row in df.iterrows():
    if row["BMI"] < 18.5:
        df.at[i, "c_weight"] = 1
    elif row["BMI"] < 25:
        df.at[i, "c_weight"] = 2
    elif row["BMI"] < 30:
        df.at[i, "c_weight"] = 3
    else:
        df.at[i, "c_weight"] = 4

In [ ]:
for i, row in df.iterrows():
    if row["ap_hi"] < 120:
        df.at[i, "c_ap_hi"] = 1
    elif row["ap_hi"] < 130:
        df.at[i, "c_ap_hi"] = 2
    else:
        df.at[i, "c_ap_hi"] = 3

In [ ]:
for i, row in df.iterrows():
    if row["ap_lo"] < 80:
        df.at[i, "c_ap_lo"] = 1
    elif row["ap_lo"] < 90:
        df.at[i, "c_ap_lo"] = 2
    else:
        df.at[i, "c_ap_lo"] = 3

In [ ]:
df = df.drop(["age", "ap_hi", "ap_lo", "BMI", "height", "weight", "id"], axis=1)

In [ ]:
df["gender"].value_counts(normalize=True)

In [ ]:
df["alco"].value_counts(normalize=True)

In [ ]:
df["smoke"].value_counts(normalize=True)

In [ ]:
df["active"].value_counts(normalize=True)

In [ ]:
df =df.astype("category")

In [ ]:
df = df.drop(["gender", "alco", "c_ap_lo", "gluc", "active"], axis=1)

In [ ]:
def likelihood(variables, df):
    df2 = df.groupby(list(df.columns)).size().reset_index().rename(columns={0:'count'})
    df2["count"] = df2["count"]/sum(df2["count"])
    for i, row in df2.iterrows():
        if all(row[0:len(df.columns)]==variables):
            return row[len(df.columns)]

In [ ]:
from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
import math
def structure(variables, df):
    parents = []
    maximum = None
    for candidate in powerset(variables):
        if len(candidate) == 0:
            continue
        score = 0
        for i, row in df.iterrows():
            score += math.log2(likelihood(row[list(candidate)], df[list(candidate)]))
        dim = 0
        for attribute in candidate:
            dim += len(list(df[attribute].values.categories))
        score -= 0.5 * math.log2(df.size) * dim
        if maximum == None or score > maximum:
            maximum = score
            parents = candidate
    return parents

In [ ]:
subset_length = 2000
structure(list(df.columns), df[:subset_length])